# HTTPRouter using Trie data structure

## Problem

Implement an HTTPRouter like found in a typical web server using the Trie data structure.

There are many different implementations of HTTP Routers such as regular expressions or simple string matching, but the Trie is an excellent and very efficient data structure for this purpose.

The purpose of an HTTP Router is to take a URL path like "/", "/about", or "/blog/2019-01-15/my-awesome-blog-post" and figure out what content to return. In a dynamic web server, the content will often come from a block of code called a handler.

First we need to implement a slightly different Trie than the one we used for autocomplete. Instead of simple words the Trie will contain a part of the http path at each node, building from the root node /

In addition to a path though, we need to know which function will handle the http request. In a real router we would probably pass an instance of a class like Python's SimpleHTTPRequestHandler which would be responsible for handling requests to that path. For the sake of simplicity we will just use a string that we can print out to ensure we got the right handler

We could split the path into letters similar to how we did the autocomplete Trie, but this would result in a Trie with a very large number of nodes and lengthy traversals if we have a lot of pages on our site. A more sensible way to split things would be on the parts of the path that are separated by slashes ("/"). A Trie with a single path entry of: "/about/me" would look like:

(root, None) -> ("about", None) -> ("me", "About Me handler")


## Code


In [75]:
# A RouteTrie will store our routes and their associated handlers
class RouteTrie:
    def __init__(self, handler):
        self.root = RouteTrieNode(handler)
        
    def insert(self, path, handler=''):
       
        current_node = self.root
        
        for part in path:
            if part not in current_node.children:
                current_node.insert(part)
            current_node = current_node.children[part]
        current_node.EOW = True
        current_node.handler = handler
        
    def find(self, path):

        current_node = self.root
        
        for part in path:
            
            if part not in current_node.children:
                return None 
            current_node = current_node.children[part]
            
        return current_node
        
# A RouteTrieNode will be similar to our autocomplete TrieNode... with one additional element, a handler.
class RouteTrieNode:
    def __init__(self, handler=None):
        self.children = {}
        self.EOW = False # EOW = end of word 
        self.handler = handler

    def insert(self, part=''):
        self.children[part] = RouteTrieNode()

In [78]:
# The Router class will wrap the Trie and handle 
class Router:
    def __init__(self, root_handler, not_found_handler):
        
        self.RouteTrie = RouteTrie(root_handler)
        self.not_found_handler = not_found_handler
       
        
    def add_handler(self, path, handler):
     
        sp_path = self.split_path(path)
        self.RouteTrie.insert(sp_path, handler)
        

    def lookup(self, path):
     
        sp_path = self.split_path(path)
        result = self.RouteTrie.find(sp_path)
        
        if result is None:
            ret = self.not_found_handler
        else:
            ret = result.handler
        
        if ret is None:
            ret = self.not_found_handler
        
        return ret
        

    def split_path(self, path):
       
        sp_path = path.split('/')
        sp_path = [x for x in sp_path if len(x) > 0]
        
        return sp_path 

## Test Cases

In [79]:
# create the router and add a route
router = Router("root handler", "not found handler") # remove the 'not found handler' if you did not implement this
router.add_handler("/home/about", "about handler")  # add a route

# some lookups with the expected output
print(router.lookup("/")) # should print 'root handler'
print(router.lookup("/home")) # should print 'not found handler' or None if you did not implement one
print(router.lookup("/home/about")) # should print 'about handler'
print(router.lookup("/home/about/")) # should print 'about handler' or None if you did not handle trailing slashes
print(router.lookup("/home/about/me")) # should print 'not found handler' or None if you did not implement one

root handler
not found handler
about handler
about handler
not found handler


## Comment

This uses Trie data structure and the time complexity is O(n), where n is the length of the parts of the url